In [1]:
#Import packages
import pandas as pd

In [47]:
#Set session path
location=input("Where are you working today? (Options: 'locally', 'SOL')")

if location=='locally':
    path='/Users/cameronkelsey/Documents/smack_lab/cayo_data/'
elif location=='SOL':
    path='/scratch/ckelsey4/Cayo_meth'

#Load in metadata
metadata=pd.read_table(f'{path}metadata_temp_clean_241106.txt', sep=r'\s+')

#Subset metadata to whole blood
blood_metadata=metadata[metadata['grantparent_tissueType'] == 'whole_blood'].dropna(subset='age_at_sampling')

In [48]:
#Extract pid from lid_pid column and move pid column next to lid column
blood_metadata['pid']=blood_metadata.lid_pid.str.extract('(PID_.*)')

col_to_move=blood_metadata.pop("pid")
lid_loc=blood_metadata.columns.get_loc("lid")+1
blood_metadata.insert(lid_loc, "pid", col_to_move)


In [49]:
#Generate column for university batch effects based on prep_date
blood_metadata['university']='uw'
blood_metadata.loc[blood_metadata['prep_date'] > '2020-01-01', 'university']='asu'


In [ ]:
#Generate longitudinal metadata df
ids_to_keep=blood_metadata.groupby('monkey_id').size().reset_index(name='counts')
long_data=blood_metadata[blood_metadata['monkey_id'].isin(ids_to_keep[ids_to_keep['counts']>1]['monkey_id'])]

long_data=long_data.copy()

#Generate within.age and between.age cols
long_data.loc[:, 'between.age']=long_data.groupby('monkey_id')['age_at_sampling'].transform('mean')
long_data.loc[:, 'within.age']=long_data.groupby('monkey_id')['age_at_sampling'].transform(lambda x: x - x.mean())

#At this point the longitudinal df has duplicates for some monkey_ids where the age_at_sampling is the same.
#The select_lids.R script will deal with this by picking the lid_pid with the largest m/cov 

In [ ]:
blood_metadata.to_csv(f'{path}blood_metadata_full.txt', sep='\t', index=False, header=True)
long_data.to_csv(f'{path}long_data.txt', sep='\t', index=False, header=True)